Тема «POS-tagger и NER»

Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы  


Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


Задание 1.

In [1]:
!pip install pyconll

In [2]:
!pip install -U spacy
!python -m spacy info
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 12.2/12.2 MB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 4.1 MB/s eta 0:00:00
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
     ---------------------------------------- 48.9/48.9 kB 2.4 MB/s eta 0:00:00
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
     -------------------------------------- 480.9/480.9 kB 4.3 MB/s eta 0:00:00
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
     ---------------------------------------- 2.1/2.1 MB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 94.7/94.7 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 4.5 MB/s eta 0:00:00

============================== Info about spaCy ==============================

spaCy version    3.5.4                         


In [3]:
import pyconll

import pandas as pd

import nltk
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [7]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [8]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [9]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [10]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [11]:
fdata_train[:3]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

In [12]:
unigram_tagger = UnigramTagger(fdata_train)
acc_u = unigram_tagger.evaluate(fdata_test)
display(acc_u)

C:\Users\rosec\AppData\Local\Temp\ipykernel_2516\4208555745.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_u = unigram_tagger.evaluate(fdata_test)


0.823732013802982

In [13]:
bigram_tagger = BigramTagger(fdata_train)
acc_b = bigram_tagger.evaluate(fdata_test)
display(acc_b)

C:\Users\rosec\AppData\Local\Temp\ipykernel_2516\2907996023.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_b = bigram_tagger.evaluate(fdata_test)


0.6093886320724006

In [14]:
trigram_tagger = TrigramTagger(fdata_train)
acc_t = trigram_tagger.evaluate(fdata_test)
display(acc_t)

C:\Users\rosec\AppData\Local\Temp\ipykernel_2516\678438604.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_t = trigram_tagger.evaluate(fdata_test)


0.1778631421316492

In [15]:
combination_1 = [BigramTagger, TrigramTagger]
combination_2 = [TrigramTagger, UnigramTagger]
combination_3 = [UnigramTagger, BigramTagger]
combination_4 = [BigramTagger, TrigramTagger, UnigramTagger]

In [16]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

acc_bo = []

for combination in [combination_1, combination_2, combination_3, combination_4]:
    backoff = nltk.DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         combination,  
                         backoff = backoff) 

    acc_bo.append(tag.evaluate(fdata_test))

acc_bo

C:\Users\rosec\AppData\Local\Temp\ipykernel_2516\2746214562.py:14: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  acc_bo.append(tag.evaluate(fdata_test))


[0.1750309264926102,
 0.21248779217396965,
 0.8275343446838986,
 0.3623152548994075]

In [17]:
result = pd.DataFrame({'Tagger': ['UnigramTagger', 'BigramTagger', 'TrigramTagger', 'BigramTagger & TrigramTagger', 'TrigramTagger & UnigramTagger', 'UnigramTagger & BigramTagger', 'All taggers'], 'Accuracy' : [acc_u, acc_b, acc_t] + acc_bo})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
5,UnigramTagger & BigramTagger,0.827534
0,UnigramTagger,0.823732
1,BigramTagger,0.609389
6,All taggers,0.362315
4,TrigramTagger & UnigramTagger,0.212488
2,TrigramTagger,0.177863
3,BigramTagger & TrigramTagger,0.175031


In [18]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [19]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [20]:
vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
hvectorizer = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
hvectorizer_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]

In [21]:
f1_scores = []
acc_scores = []

for vectorizer in vectorizers + vectorizers_word + hvectorizer + hvectorizer_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    acc_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

C:\Users\rosec\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

C:\Users\rosec\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.90      0.91      0.91     11247
         ADP       0.99      0.99      0.99     10255
         ADV       0.92      0.87      0.89      5986
         AUX       0.81      0.97      0.89      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.80      0.83      0.82      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.90      0.96      0.93     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.85      0.90      0.87      1436
        PART       0.95      0.79      0.86      3762
        PRON       0.87      0.84      0.86      5346
       PROPN       0.80      0.52      0.63      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.64      0.78        53
        VERB       0.93     

C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rosec\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alt

...

C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rosec\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

HashingVectorizer(n_features=5000, ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.63      0.33      0.44     11247
         ADP       0.91      0.48      0.63     10255
         ADV       0.82      0.72      0.77      5986
         AUX       0.79      0.95      0.86      1058
       CCONJ       0.92      0.19      0.31      4276
         DET       0.69      0.70      0.70      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.70      0.61      0.65     27241
      NO_TAG       0.00      0.00      0.00       197
         NUM       0.66      0.48      0.55      1436
        PART       0.91      0.76      0.83      3762
        PRON       0.79      0.79      0.79      5346
       PROPN       0.54      0.13      0.21      4315
       PUNCT       0.44      1.00      0.62     21941
       SCONJ       0.76      0.88      0.82      2176
         SYM       0.00      0.00      0.00        53
        VERB       0.67   

C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rosec\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

HashingVectorizer(n_features=10000, ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.73      0.36      0.48     11247
         ADP       0.95      0.48      0.64     10255
         ADV       0.88      0.74      0.81      5986
         AUX       0.81      0.88      0.84      1058
       CCONJ       0.88      0.20      0.33      4276
         DET       0.72      0.75      0.74      2978
        INTJ       0.00      0.00      0.00        11
        NOUN       0.77      0.64      0.70     27241
      NO_TAG       0.00      0.00      0.00       197
         NUM       0.73      0.57      0.64      1436
        PART       0.96      0.75      0.84      3762
        PRON       0.83      0.78      0.81      5346
       PROPN       0.67      0.15      0.24      4315
       PUNCT       0.42      1.00      0.59     21941
       SCONJ       0.78      0.88      0.83      2176
         SYM       0.00      0.00      0.00        53
        VERB       0.76  

C:\Users\rosec\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + hvectorizer + hvectorizer_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927836
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921185
9,"HashingVectorizer(analyzer='char', n_features=...",0.903654
8,"HashingVectorizer(analyzer='char', n_features=...",0.901786
7,"HashingVectorizer(analyzer='char', n_features=...",0.896609
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882224
4,"TfidfVectorizer(ngram_range=(1, 3))",0.663954
3,"CountVectorizer(ngram_range=(1, 3))",0.649238
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.618026


In [23]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + hvectorizer + hvectorizer_word,
                            'Accuracy': acc_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929513
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923609
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905487
7,"HashingVectorizer(analyzer='char', n_features=...",0.900330
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885165
4,"TfidfVectorizer(ngram_range=(1, 3))",0.654452
3,"CountVectorizer(ngram_range=(1, 3))",0.640852
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.627357


На первом месте Accuracy коллаборация триггеров из UnigramTagger & BigramTagger с лучшим результатом - 0.93 Во-втором и в-третьем тестах лидирует CountVectorizer с f1_score 0.927826 и Accuracy 0.929504.

Задание 2.

In [24]:
!pip install navec
!pip install slovnet
!pip install ipymarkup
!pip install sklearn_crfsuite
!pip install patool

     -------------------------------------- 46.7/46.7 kB 774.9 kB/s eta 0:00:00
  Using cached razdel-0.5.0-py3-none-any.whl (21 kB)
     -------------------------------------- 139.4/139.4 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 77.5/77.5 kB 1.4 MB/s eta 0:00:00


In [86]:
import os
import glob

import nltk
import csv
from razdel import tokenize
import pandas as pd

from spacy import displacy
import ru_core_news_sm

from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup

import sklearn_crfsuite
from sklearn_crfsuite import metrics

In [50]:
import patoolib
patoolib.extract_archive("collection3.zip", outdir="collection3")

patool: Extracting collection3.zip ...
patool: ... collection3.zip extracted to `collection3'.


'collection3'

In [99]:
documents_txt = [_ for _ in os.listdir("collection3/Collection3/") if _.endswith(".txt")]
text_list = []
for file in documents_txt:
    doc = open('collection3/Collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)

In [100]:
data_text = pd.DataFrame({'text': text_list})
data_text

,text
0,Россия рассчитывает на конструктивное воздейст...
1,Комиссар СЕ критикует ограничительную политику...
2,"Пулеметы, автоматы и снайперские винтовки изъя..."
3,4 октября назначены очередные выборы Верховног...
4,Следственное управление при прокуратуре требуе...
...,...
995,"Депутат от ""ЕР"": К отставке А.Сердюкова причас..."
996,\nСи Цзиньпин избран генсеком Коммунистической...
997,"""Ведомости"" узнали о смене лидера московских е..."
998,СМИ узнали о кутежах туркменского чиновника на...


In [101]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rosec\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rosec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [102]:
document = data_text.text[0]
nltk.pos_tag(nltk.word_tokenize(document))

[('Россия', 'JJ'),
 ('рассчитывает', 'NNP'),
 ('на', 'NNP'),
 ('конструктивное', 'NNP'),
 ('воздействие', 'NNP'),
 ('США', 'NNP'),
 ('на', 'NNP'),
 ('Грузию', 'VBD'),
 ('04/08/2008', 'CD'),
 ('12:08', 'CD'),
 ('МОСКВА', 'NN'),
 (',', ','),
 ('4', 'CD'),
 ('авг', 'SYM'),
 ('-', ':'),
 ('РИА', 'NN'),
 ('Новости', 'NN'),
 ('.', '.'),
 ('Россия', 'JJ'),
 ('рассчитывает', 'NN'),
 (',', ','),
 ('что', 'NNP'),
 ('США', 'NNP'),
 ('воздействуют', 'NNP'),
 ('на', 'NNP'),
 ('Тбилиси', 'NNP'),
 ('в', 'NNP'),
 ('связи', 'NNP'),
 ('с', 'NNP'),
 ('обострением', 'NNP'),
 ('ситуации', 'NNP'),
 ('в', 'NNP'),
 ('зоне', 'NNP'),
 ('грузино-осетинского', 'JJ'),
 ('конфликта', 'NNP'),
 ('.', '.'),
 ('Об', 'VB'),
 ('этом', 'JJ'),
 ('статс-секретарь', 'JJ'),
 ('-', ':'),
 ('заместитель', 'NN'),
 ('министра', 'JJ'),
 ('иностранных', 'NNP'),
 ('дел', 'NNP'),
 ('России', 'NNP'),
 ('Григорий', 'NNP'),
 ('Карасин', 'NNP'),
 ('заявил', 'NNP'),
 ('в', 'NNP'),
 ('телефонном', 'NNP'),
 ('разговоре', 'NNP'),
 ('с', 'NNP

In [103]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\rosec\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\rosec\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [104]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('МИД России', 'ORGANIZATION'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('России Григорий Карасин', 'PERSON'),
 ('Россия', 'PERSON'),
 ('Тбилиси', 'PERSON')}

In [119]:
pd.read_csv('collection3/Collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


In [120]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [121]:
for token in article:
    print(token.text, token.pos_, token.dep_)

Россия PROPN nsubj
рассчитывает VERB ROOT
на ADP case
конструктивное ADJ amod
воздействие NOUN obl
США PROPN nmod
на ADP case
Грузию PROPN nmod


 SPACE dep
04/08/2008 PROPN appos
12:08 NUM appos


 SPACE dep
МОСКВА PROPN nmod
, PUNCT punct
4 NUM conj
авг PROPN obl
- NOUN obl
РИА PROPN obl
Новости PROPN appos
. PUNCT punct
Россия PROPN nsubj
рассчитывает VERB ROOT
, PUNCT punct
что SCONJ mark
США PROPN nsubj
воздействуют VERB ccomp
на ADP case
Тбилиси PROPN obl
в ADP case
связи NOUN fixed
с ADP fixed
обострением NOUN obl
ситуации NOUN nmod
в ADP case
зоне NOUN nmod
грузино ADJ amod
- ADJ amod
осетинского ADJ amod
конфликта NOUN nmod
. PUNCT punct
Об ADP case
этом PRON obl
статс NOUN nsubj
- NOUN nsubj
секретарь NOUN nsubj
- NOUN nsubj
заместитель NOUN nsubj
министра NOUN nmod
иностранных ADJ amod
дел NOUN nmod
России PROPN nmod
Григорий PROPN appos
Карасин PROPN flat:name
заявил VERB ROOT
в ADP case
телефонном ADJ amod
разговоре NOUN obl
с ADP case
заместителем NOUN nmod
госсекретаря N

In [122]:
pd.read_csv('collection3/Collection3/001.ann', delimiter='\t', header=None)

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [126]:
text = data_text.text[0]
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

Россия рассчитывает на конструктивное воздействие США на Грузию
LOC───                                            LOC    LOC───
04/08/2008 12:08
МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют
LOC───          ORG────────  LOC───                   LOC             
 на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского 
    LOC────                                                           
конфликта. Об этом статс-секретарь - заместитель министра иностранных 
дел России Григорий Карасин заявил в телефонном разговоре с 
    LOC─── PER─────────────                                 
заместителем госсекретаря США Дэниэлом Фридом.
                          LOC PER──────────── 
"С российской стороны выражена глубокая озабоченность в связи с новым 
витком напряженности вокруг Южной Осетии, противозаконными действиями 
                            LOC─────────                              
грузинской стороны по наращиванию своих вооруженных сил в регионе, 
бес

In [140]:
fileDir = r"collection3/Collection3/"
fileExt = r".ann"
documents_ann = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_ann)

['001.ann', '002.ann', '003.ann', '004.ann', '005.ann', '006.ann', '007.ann', '008.ann', '009.ann', '010.ann', '011.ann', '012.ann', '013.ann', '014.ann', '015 (!).ann', '016.ann', '017.ann', '018.ann', '019.ann', '020.ann', '021.ann', '022.ann', '023.ann', '025.ann', '026.ann', '027.ann', '028.ann', '029.ann', '030.ann', '031.ann', '032.ann', '033.ann', '034.ann', '035.ann', '036.ann', '037.ann', '038.ann', '039.ann', '03_12_12a.ann', '03_12_12b.ann', '03_12_12c.ann', '03_12_12d.ann', '03_12_12g.ann', '03_12_12h.ann', '040.ann', '041.ann', '042.ann', '043.ann', '044.ann', '045.ann', '046.ann', '047.ann', '048.ann', '049.ann', '04_02_13a_abdulatipov.ann', '04_03_13a_sorokin.ann', '04_12_12b.ann', '04_12_12d.ann', '04_12_12f.ann', '04_12_12g.ann', '04_12_12h_corr.ann', '050.ann', '051.ann', '052.ann', '053.ann', '054.ann', '055.ann', '056.ann', '057.ann', '058.ann', '059.ann', '060.ann', '061.ann', '062.ann', '063.ann', '064.ann', '065.ann', '066.ann', '067.ann', '068.ann', '069.ann', '

In [141]:
ann = pd.read_csv('collection3/Collection3/001.ann', delimiter='\t', header=None)
ann

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [142]:
docs = []
# print(len(data_text))
# print(len(documents_ann))
# for i in range(len(data_text)):
for i in range(len(documents_ann)):
    words = []
    labels = []
    text = data_text['text'][i]
    # print(documents_ann[i])
    df = pd.read_csv('collection3/Collection3/' + documents_ann[i], delimiter='\t', header=None, quoting=csv.QUOTE_NONE, encoding='utf-8')
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1].values]
    df_ann['Entity'] = [loc[0] for loc in split_1]
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        entity = df_ann['Entity'][j]
        for tok in token:
            dic[tok] = entity

    for token in tokenize(text):
        if (token.text.lower() in dic.keys()):
            words.append(token.text)
            labels.append(dic[token.text.lower()])
        else:
            words.append(token.text)
            labels.append('OUT')
    
    docs.append([words, labels])

In [143]:
print(docs[0][0]), print(docs[0][1])

['Россия', 'рассчитывает', 'на', 'конструктивное', 'воздействие', 'США', 'на', 'Грузию', '04/08/2008', '12', ':', '08', 'МОСКВА', ',', '4', 'авг', '-', 'РИА', 'Новости', '.', 'Россия', 'рассчитывает', ',', 'что', 'США', 'воздействуют', 'на', 'Тбилиси', 'в', 'связи', 'с', 'обострением', 'ситуации', 'в', 'зоне', 'грузино-осетинского', 'конфликта', '.', 'Об', 'этом', 'статс-секретарь', '-', 'заместитель', 'министра', 'иностранных', 'дел', 'России', 'Григорий', 'Карасин', 'заявил', 'в', 'телефонном', 'разговоре', 'с', 'заместителем', 'госсекретаря', 'США', 'Дэниэлом', 'Фридом', '.', '"', 'С', 'российской', 'стороны', 'выражена', 'глубокая', 'озабоченность', 'в', 'связи', 'с', 'новым', 'витком', 'напряженности', 'вокруг', 'Южной', 'Осетии', ',', 'противозаконными', 'действиями', 'грузинской', 'стороны', 'по', 'наращиванию', 'своих', 'вооруженных', 'сил', 'в', 'регионе', ',', 'бесконтрольным', 'строительством', 'фортификационных', 'сооружений', '"', ',', '-', 'говорится', 'в', 'сообщении', '

(None, None)

In [144]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Россия	LOC
рассчитывает	OUT
на	OUT
конструктивное	OUT
воздействие	OUT
США	LOC
на	OUT
Грузию	LOC
04/08/2008	OUT
12	OUT
:	OUT
08	OUT
МОСКВА	LOC
,	OUT
4	OUT
авг	OUT
-	OUT
РИА	ORG
Новости	ORG
.	OUT
Россия	LOC
рассчитывает	OUT
,	OUT
что	OUT
США	LOC
воздействуют	OUT
на	OUT
Тбилиси	LOC
в	OUT
связи	OUT
с	OUT
обострением	OUT
ситуации	OUT
в	OUT
зоне	OUT
грузино-осетинского	OUT
конфликта	OUT
.	OUT
Об	OUT
этом	OUT
статс-секретарь	OUT
-	OUT
заместитель	OUT
министра	OUT
иностранных	OUT
дел	OUT
России	LOC
Григорий	PER
Карасин	PER
заявил	OUT
в	OUT
телефонном	OUT
разговоре	OUT
с	OUT
заместителем	OUT
госсекретаря	OUT
США	LOC
Дэниэлом	PER
Фридом	PER
.	OUT
"	OUT
С	OUT
российской	OUT
стороны	OUT
выражена	OUT
глубокая	OUT
озабоченность	OUT
в	OUT
связи	OUT
с	OUT
новым	OUT
витком	OUT
напряженности	OUT
вокруг	OUT
Южной	LOC
Осетии	LOC
,	OUT
противозаконными	OUT
действиями	OUT
грузинской	OUT
стороны	OUT
по	OUT
наращиванию	OUT
своих	OUT
вооруженных	OUT
сил	OUT
в	OUT
регионе	OUT
,	OUT
бесконтрольным	OUT
строительс

In [145]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,Россия,LOC
1,0,рассчитывает,OUT
2,0,на,OUT
3,0,конструктивное,OUT
4,0,воздействие,OUT
5,0,США,LOC
6,0,на,OUT
7,0,Грузию,LOC
8,0,04/08/2008,OUT
9,0,12,OUT


In [146]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)

sentences = getter.sentences

In [147]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [148]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [149]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X[2][7]

{'bias': 1.0,
 'word.lower()': 'в',
 'word[-3:]': 'в',
 'word[-2:]': 'в',
 'word.isdigit()': False,
 '-1:word.lower()': 'изъяты',
 '+1:word.lower()': 'арендуемом'}

In [150]:
len(X)

1000

In [151]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [152]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', # Градиентный спуск с использованием метода L-BFGS
    c1=0.1, # Коэффициент для регуляризации L1
    c2=0.1, # Коэффициент для регуляризации L2
    max_iterations=1000, # Максимальное количество итераций
    all_possible_transitions=True, # Генерация объектов (не встречающихся в обучающих данных)
    verbose=False # Выключение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

CPU times: total: 16.3 s
Wall time: 28.8 s


In [153]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [154]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9464472957366774

In [155]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [156]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X[1][7]

{'bias': 1.0,
 'word.lower()': 'беженцев',
 'word[-3:]': 'цев',
 'word[-2:]': 'ев',
 'word.isdigit()': False}

In [157]:
len(X)

1000

In [158]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [159]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=False
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

CPU times: total: 15.6 s
Wall time: 26.1 s


In [160]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [161]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9414616794196085